In [0]:
#Get Data from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
%cd Dropbox-Uploader
!chmod +x dropbox_uploader.sh
!bash dropbox_uploader.sh
!echo "CHTZbyeOjyYAAAAAAAAAfZTfC_d3kTmHXSqg7-941o9Nolj-FEAJbjyUKutEtCSu" > token.txt
!bash dropbox_uploader.sh

!bash dropbox_uploader.sh download cola.zip
!unzip cola.zip
!rm cola.zip



Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 309.19 KiB | 7.73 MiB/s, done.
Resolving deltas: 100% (505/505), done.
/content/Dropbox-Uploader

 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader17239344619851)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt


 This is the first time you run thi

In [2]:
!pip install pytorch-nlp
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix

from torchnlp.datasets import smt_dataset
from torchnlp.datasets import imdb_dataset



     |████████████████████████████████| 92kB 5.6MB/s 


In [0]:
def restrict_data_func(X_train, Y_train, num_each_label, num_labels):
  

  X_train_labelled = []
  Y_train_labelled = []

  for i in range(num_labels):
      X_train_labelled.append(X_train[Y_train == i][:num_each_label])
      Y_train_labelled.append(Y_train[Y_train == i][:num_each_label])
  X_train_labelled = np.concatenate(X_train_labelled, axis=0)
  Y_train_labelled = np.concatenate(Y_train_labelled, axis=0)

  return X_train_labelled, Y_train_labelled

In [0]:
#FUNCTION TO CREATE LABELS FOR SMT
def create_SMT_labels(data,num_train):
  labels_tensor = np.zeros(num_train)
  labels = data.__getitem__('label')
  labels = labels[0:num_train]
  
  very_pos_indices = [i for i, x in enumerate(labels) if x == "very positive"]
  pos_indices = [i for i, x in enumerate(labels) if x == "positive"]
  neut_indices = [i for i, x in enumerate(labels) if x == "neutral"]
  neg_indices = [i for i, x in enumerate(labels) if x == "negative"]
  very_neg_indices = [i for i, x in enumerate(labels) if x == "very negative"]

  labels_tensor[very_pos_indices] = 0
  labels_tensor[pos_indices] = 1
  labels_tensor[neut_indices] = 2
  labels_tensor[neg_indices] = 3
  labels_tensor[very_neg_indices] = 4
  
  return labels_tensor

#FUNCTION TO CREATE LABELS FOR IMDB
def create_IMDB_labels(data,num_train):
  labels_tensor = np.zeros(num_train)
  labels = data.__getitem__('sentiment')
  labels = labels[0:num_train]

  pos_indices = [i for i, x in enumerate(labels) if x == "pos"]
  neg_indices = [i for i, x in enumerate(labels) if x == "neg"]

  labels_tensor[pos_indices] = int(1)
  labels_tensor[neg_indices] = int(0)
  
  return labels_tensor

In [5]:
#Load preprocessed Data

"""
train_labels = np.load('cola/cola_train_labels.npy')
train_text = np.load('cola/cola_train_text.npy')
valid_labels = np.load('cola/cola_in_domain_dev_labels.npy')
valid_text = np.load('cola/cola_in_domain_dev_text.npy')
test_labels = np.load('cola/cola_out_domain_dev_labels.npy')
test_text = np.load('cola/cola_out_domain_dev_text.npy')

"""

train = imdb_dataset(train=True)
test = imdb_dataset(test=True)
train_labels = create_IMDB_labels(train,25000)
train_text = np.array(train.__getitem__('text'))
test_labels = create_IMDB_labels(test,25000)
test_text = np.array(test.__getitem__('text'))


"""


train = smt_dataset(train=True, fine_grained=True)
valid = smt_dataset(dev=True, fine_grained=True)
test = smt_dataset(test=True, fine_grained=True)

num_train = len(train)
num_valid = len(valid)
num_test = len(test)


train_labels = create_SMT_labels(train,num_train)
train_text = np.array(train.__getitem__('text'))
test_labels = create_SMT_labels(test,num_test)
test_text = np.array(test.__getitem__('text'))
valid_labels = create_SMT_labels(valid,num_valid)
valid_text = np.array(valid.__getitem__('text'))

"""

aclImdb_v1.tar.gz: 84.1MB [00:01, 45.0MB/s]                            


"\n\n\ntrain = smt_dataset(train=True, fine_grained=True)\nvalid = smt_dataset(dev=True, fine_grained=True)\ntest = smt_dataset(test=True, fine_grained=True)\n\nnum_train = len(train)\nnum_valid = len(valid)\nnum_test = len(test)\n\n\ntrain_labels = create_SMT_labels(train,num_train)\ntrain_text = np.array(train.__getitem__('text'))\ntest_labels = create_SMT_labels(test,num_test)\ntest_text = np.array(test.__getitem__('text'))\nvalid_labels = create_SMT_labels(valid,num_valid)\nvalid_text = np.array(valid.__getitem__('text'))\n\n"

In [6]:
#DECIDE HOW MANY OF EACH CLASS TO USE IN TRAINING DATA
restrict_training_data = 0 #Set equal to 0 or 1 
num_each_label=100
num_labels = 2


#Shuffle Data
random_seed = 1
rand_state_1 = np.random.RandomState(random_seed)
shuffle_1 = rand_state_1.permutation(train_text.shape[0])
train_text = train_text[shuffle_1]
train_labels = train_labels[shuffle_1]



if restrict_training_data:
  train_text, train_labels = restrict_data_func(train_text, train_labels, num_each_label,num_labels)



print(len(train_text))
print(len(train_labels))


25000
25000


In [9]:

  
nb = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer(sublinear_tf=True)),
               ('clf', MultinomialNB(alpha=1)),
              ])

nb.fit(train_text, train_labels)

y_pred = nb.predict(test_text)


print('accuracy %s' % accuracy_score(y_pred, test_labels))




accuracy 0.87332


In [10]:
y_pred = nb.predict(test_text)

print('accuracy %s' % accuracy_score(y_pred, test_labels))


accuracy 0.87332


In [0]:
#y_pred = nb.predict(valid_text)
#print('accuracy %s' % accuracy_score(y_pred, valid_labels))